<a href="https://colab.research.google.com/github/ev1025/DA_Study/blob/main/Ga%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openpyxl

In [ ]:
pip install ipywidgets

In [ ]:
import pandas as pd
from pandas import json_normalize
import json
import os
import csv
import re
import ast
import copy
from tqdm.notebook import tqdm
from datetime import datetime


In [ ]:
# train_v2가 있는 경로로 이동
# os.chdir('D:\\train_v2.csv')

**데이터 가져오기**

- 데이터 대용량(25GB)이슈로 인하여 read_csv의 chunksize를 사용하여 30만개의 데이터만 추출
- 확인결과 2016년 10월 ~ 2018년 4월 데이터 추출
- 데이터 용량이 4GB로 축소하였으나, 태블로 사용시 권장 용량이 4GB미만인 점과 1년의 추세를 보고자 2017년 자료로 사용

In [ ]:
# # address에 chunk 저장
# address = pd.read_csv('train_v2.csv', chunksize=10000,
#                        dtype={'fullVisitorId':'str',('date','fulVisitorid', 'visitId', 'visitNumber', 'visitStartTime'):'int'})
# # 빈프레임 생성
# train_data = pd.DataFrame()

In [ ]:
# # 1만개씩 분할하여 빈 프레임에 30만개 수집
# i = 0
# for row, chunk in enumerate(address):
#     train_data = pd.concat([train_data, chunk])
#     print(f'{row*10000}개')
#     i+=1
#     if i == 30:
#         break

In [ ]:
# # 2017년 GA쇼핑몰데이터만 저장
# ga_2017 = train_data[train_data['date'].astype('str').str.contains('2017')]
# ga_2017.to_csv('2017_ga_data.csv', index=False, lineterminator='\n')

In [ ]:
# df = pd.read_csv('2017_ga_data.csv', 
#                 dtype={'fullVisitorId': 'str'})

In [ ]:
# df2 = df.copy()

- 'hits' 컬럼에 read_csv의 converter가 적용되지 않는 이슈가 발생
- 원인을 찾지 못해서 따로 json.loads 진행해주기로 결정
- ' '로 묶인 문자는 json에서 사용할 수 없다는 오류 발생하여 '를 "로 변경
- Bool형식을 인식하지 못해서 True => "True" , False => "False"로 변경
- ' 을 "로 바꾸는 과정에서 제품명에 Men's와 같은 문자가 포함되어 있어서 문자열이 끊어지는 현상 발생
- 결국 ast.literal_eval() 함수를 apply하여 json형식으로 변경

In [ ]:
## 각 컬럼의 json을 활성화(str=>json)
# JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource','hits']

# for i in JSON_COLUMNS:
#     df2[i] = df2[i].apply(lambda x : ast.literal_eval(x))
#     print(f'{i} 끝')

In [ ]:
# # json형태의 컬럼들을  key : value 형태로 컬럼화
# json_to_col = ['device','geoNetwork', 'totals', 'trafficSource']
# for column in json_to_col:
#         column_as_df = json_normalize(df2[column])
#         column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
#         df2 = df2.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
# df2.shape

In [ ]:
# os.chdir('C:\\Users\\ENVY\\Desktop')
# df2 = pd.read_csv('json_convert.csv')

C:\Users\ENVY\AppData\Local\Temp\ipykernel_7224\3722996920.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('json_convert.csv')


- customDimensions와 hits컬럼은 이중 json이라 따로 정리

In [ ]:
# 이중 dict형태라서 따로 정리
tqdm.pandas()
df2['customDimensions'] = df2['customDimensions'].progress_apply(lambda x : ast.literal_eval(x))

In [ ]:
custom = json_normalize(df2['customDimensions'])
cus_col = json_normalize(custom[0])
cus_col.columns = [f'{"customDimensions"}.{subcolumn}' for subcolumn in cus_col.columns]
df2 = df2.drop(['customDimensions'],axis=1).merge(cus_col,right_index=True, left_index=True)

In [ ]:
# 조회한 횟수 만큼 컬럼이 생김
hit = df2['hits'].progress_apply(lambda x : ast.literal_eval(x))

In [ ]:
# 빈 컬럼과 데이터 컬럼 생성(hit수 만큼 컬럼이 생김)
emp_data = pd.DataFrame()
hits_col = json_normalize(hit)
hits_col.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,"{'hitNumber': '1', 'time': '0', 'hour': '17', ...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'hitNumber': '1', 'time': '0', 'hour': '10', ...","{'hitNumber': '2', 'time': '27844', 'hour': '1...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'hitNumber': '1', 'time': '0', 'hour': '17', ...","{'hitNumber': '2', 'time': '38153', 'hour': '1...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'hitNumber': '1', 'time': '0', 'hour': '9', '...","{'hitNumber': '2', 'time': '1087', 'hour': '9'...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'hitNumber': '1', 'time': '0', 'hour': '14', ...","{'hitNumber': '2', 'time': '52349', 'hour': '1...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
# # hitsnum별로 옆으로 이어붙이기(필요할까?)
# for i in tqdm(range(len(hits_col.columns))):
#     emp_data = pd.concat([emp_data, json_normalize(hits_col[i])], axis=1)

# emp_data.head()

In [ ]:
# 일단 hitNumber1에 대해서만 분석하기로 결정
emp_data = pd.concat([emp_data, json_normalize(hits_col[0])],axis=1)
df2 = df2.drop(['hits'],axis=1).merge(emp_data,right_index=True, left_index=True)
df2.to_csv('hits_last.csv',index=False)

In [ ]:
df2.head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,...,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Referral,20171016,8934116514970143966,Not Socially Engaged,1508176307,6,1508176307,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Direct,20171016,7992466427990357681,Not Socially Engaged,1508201613,1,1508201613,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20171016,9075655783635761930,Not Socially Engaged,1508169851,1,1508169851,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20171016,6960673291025684308,Not Socially Engaged,1508190552,1,1508190552,Chrome,not available in demo dataset,not available in demo dataset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df2 = pd.read_csv('GA_CONVERT.csv')

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df2.head(3)

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,customDimensions.index,customDimensions.value,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,referer,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,transaction.currencyCode,item.currencyCode,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,contentGroup.contentGroupUniqueViews2,page.searchKeyword,page.searchCategory,promotionActionInfo.promoIsView,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,eCommerceAction.option,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,1508198450,1,1508198450,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,EMEA,1.0,0.0,17.0,0.0,True,True,True,https://www.google.co.uk/search?q=water+bottle...,"[{'productSKU': 'GGOEGDHC074099', 'v2ProductNa...",[],[],[],[],[],PAGE,web,[],/google+redesign/bags/water+bottles+and+tumblers,shop.googlemerchandisestore.c

In [ ]:
df2['visitId'] = df2['visitId'].apply(lambda x : datetime.utcfromtimestamp(x))
df2['visitStartTime'] = df2['visitStartTime'].apply(lambda x : datetime.utcfromtimestamp(x))

In [ ]:
df2.head(3)

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,customDimensions.index,customDimensions.value,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,referer,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,transaction.currencyCode,item.currencyCode,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,contentGroup.contentGroupUniqueViews2,page.searchKeyword,page.searchCategory,promotionActionInfo.promoIsView,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,eCommerceAction.option,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
0,Organic Search,20171016,3162355547410993243,Not Socially Engaged,2017-10-17 00:00:50,1,2017-10-17 00:00:50,Firefox,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Western Europe,Germany,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,(not set),google,organic,water bottle,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,EMEA,1.0,0.0,17.0,0.0,True,True,True,https://www.google.co.uk/search?q=water+bottle...,"[{'productSKU': 'GGOEGDHC074099', 'v2ProductNa...",[],[],[],[],[],PAGE,web,[],/google+redesign/bags/water+bottles+and+tumblers,shop.google

In [ ]:
# visitid와 visitStartTime가 모두 동일한지 확인결과 숫자의 차이를 보임
print(f"visitId : {df2['visitId'].nunique()} / visitStartTime : {df2['visitStartTime'].nunique()}")

visitId : 149523 / visitStartTime : 149551


In [ ]:
df2[df2['fullVisitorId']==414992497318287346]

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,customDimensions.index,customDimensions.value,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,referer,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,transaction.currencyCode,item.currencyCode,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,contentGroup.contentGroupUniqueViews2,page.searchKeyword,page.searchCategory,promotionActionInfo.promoIsView,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,eCommerceAction.option,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
251,Referral,20171016,414992497318287346,Not Socially Engaged,2017-10-16 17:57:59,5,2017-10-16 17:57:59,Chrome,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,6,5.0,NaN,NaN,2.0,62.0,NaN,NaN,NaN,(not set),(direct),(none),NaN,not available in demo dataset,/,True,NaN,NaN,NaN,NaN,NaN,NaN,4.0,North America,1.0,0.0,10.0,57.0,True,True,NaN,https://mall.googleplex.com/,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/home,shop.googlemerchandisestore.com,Home,/home,NaN,NaN,NaN,NaN,NaN,shop.

In [ ]:
# 몇초 차이 외에는 별다른 차이를 발견하지 못함(고유id를 fullVisitorId로 하기로하고 visitId 제외)
diff_time = set(df2['visitId']).difference(set(df2['visitStartTime']))
df2[df2['visitId'].isin(diff_time)].head()

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,customDimensions.index,customDimensions.value,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,referer,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,transaction.currencyCode,item.currencyCode,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,contentGroup.contentGroupUniqueViews2,page.searchKeyword,page.searchCategory,promotionActionInfo.promoIsView,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,eCommerceAction.option,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
56,Direct,20171016,1891933966803438461,Not Socially Engaged,2017-10-16 21:45:56,1,2017-10-16 21:46:00,Safari,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,San Francisco,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,3,3.0,NaN,1.0,1.0,15.0,NaN,NaN,NaN,(not set),(direct),(none),NaN,not available in demo dataset,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,4.0,North America,2.0,0.0,14.0,46.0,True,True,NaN,https://googlemerchandisestore.com/,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/home,shop.googlemerchandisestore.com,Home,/home,NaN,NaN,NaN,NaN,NaN,shop.googlemerchandisestore.com/home,

In [ ]:
# hits컬럼의 내용들
df2.iloc[0:2,59:]

,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,referer,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,transaction.currencyCode,item.currencyCode,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,contentGroup.contentGroupUniqueViews2,page.searchKeyword,page.searchCategory,promotionActionInfo.promoIsView,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,eCommerceAction.option,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
0,1.0,0.0,17.0,0.0,True,True,True,https://www.google.co.uk/search?q=water+bottle...,"[{'productSKU': 'GGOEGDHC074099', 'v2ProductNa...",[],[],[],[],[],PAGE,web,[],/google+redesign/bags/water+bottles+and+tumblers,shop.googlemerchandisestore.com,Water Bottles & Tumblers | Drinkware | Google ...,/google+redesign/,/bags/,/water+bottles+and+tumblers,NaN,USD,USD,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,shop.googlemerchandisestore.com/google+redesig...,0.0,True,0.0,1.0,(not set),No,:,(not set),Bags,(not set),(not set),(not set),(entrance),(entrance),(entrance),(entrance),(entrance),1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,0.0,10.0,51.0,True,True,NaN,https://sites.google.com/a/google.com/transpor...,[],"[{'promoId': 'Apparel Row 1', 'promoName': 'Ap...",[],[],[],[],PAGE,web,[],/home,shop.googlemerchandisestore.com,Home,/home,NaN,NaN,NaN,NaN,NaN,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/home,shop.googlemerchandisestore.com/asearch.html,0.0,True,0.0,1.0,(not set),No,:,(not set),(not set),(not set),(not set),(not set),(entrance),(entrance),(entrance),(entrance),(entrance),NaN,jersey,(not set),True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 해당페이지에서 이탈한 경우
df2['isExit'].unique()

array([True, nan], dtype=object)

In [ ]:
# 이탈로 잡힘
df2[(df2['isExit'].isnull()) & (df2['totals.bounces']==1)][['isExit','totals.bounces']]

,isExit,totals.bounces
13542,NaN,1.0
15710,NaN,1.0
53664,NaN,1.0
60095,NaN,1.0
60218,NaN,1.0
71709,NaN,1.0
87917,NaN,1.0
109182,NaN,1.0
131731,NaN,1.0
142355,NaN,1.0


In [ ]:
# 해당페이지에서 나가지 않았는데, 이탈하지 않았다고 나오는 경우가 있음
df2[(df2['isExit'].isnull()) & (df2['totals.bounces']==1)]

,channelGrouping,date,fullVisitorId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,device.browserSize,device.operatingSystem,device.operatingSystemVersion,device.isMobile,device.mobileDeviceBranding,device.mobileDeviceModel,device.mobileInputSelector,device.mobileDeviceInfo,device.mobileDeviceMarketingName,device.flashVersion,device.language,device.screenColors,device.screenResolution,device.deviceCategory,geoNetwork.continent,geoNetwork.subContinent,geoNetwork.country,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,totals.visits,totals.hits,totals.pageviews,totals.bounces,totals.newVisits,totals.sessionQualityDim,totals.timeOnSite,totals.transactions,totals.transactionRevenue,totals.totalTransactionRevenue,trafficSource.campaign,trafficSource.source,trafficSource.medium,trafficSource.keyword,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.referralPath,trafficSource.isTrueDirect,trafficSource.adContent,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,customDimensions.index,customDimensions.value,hitNumber,time,hour,minute,isInteraction,isEntrance,isExit,referer,product,promotion,experiment,customVariables,customDimensions,customMetrics,type,dataSource,publisher_infos,page.pagePath,page.hostname,page.pageTitle,page.pagePathLevel1,page.pagePathLevel2,page.pagePathLevel3,page.pagePathLevel4,transaction.currencyCode,item.currencyCode,appInfo.screenName,appInfo.landingScreenName,appInfo.exitScreenName,appInfo.screenDepth,exceptionInfo.isFatal,eCommerceAction.action_type,eCommerceAction.step,social.socialNetwork,social.hasSocialSourceReferral,social.socialInteractionNetworkAction,contentGroup.contentGroup1,contentGroup.contentGroup2,contentGroup.contentGroup3,contentGroup.contentGroup4,contentGroup.contentGroup5,contentGroup.previousContentGroup1,contentGroup.previousContentGroup2,contentGroup.previousContentGroup3,contentGroup.previousContentGroup4,contentGroup.previousContentGroup5,contentGroup.contentGroupUniqueViews2,page.searchKeyword,page.searchCategory,promotionActionInfo.promoIsView,contentGroup.contentGroupUniqueViews1,contentGroup.contentGroupUniqueViews3,eventInfo.eventCategory,eventInfo.eventAction,eventInfo.eventLabel,eCommerceAction.option,latencyTracking.pageLoadSample,latencyTracking.pageLoadTime,latencyTracking.pageDownloadTime,latencyTracking.redirectionTime,latencyTracking.speedMetricsSample,latencyTracking.domainLookupTime,latencyTracking.serverConnectionTime,latencyTracking.serverResponseTime,latencyTracking.domLatencyMetricsSample,latencyTracking.domInteractiveTime,latencyTracking.domContentLoadedTime
13542,Referral,20170203,8645390779994242381,Not Socially Engaged,2017-02-03 20:02:48,5,2017-02-03 20:02:48,Chrome,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,(not set),(direct),(none),NaN,not available in demo dataset,/pagead/ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,North America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
df2[['fullVisitorId','date','visitStartTime','visitNumber', # 고유아이디, 날짜, 방문시작시간(세션이동시간 측정할 수 있음), 방문 횟수
     'channelGrouping','device.browser','device.operatingSystem','device.deviceCategory',
     'geoNetwork.continent','geoNetwork.subContinent', 'geoNetwork.country','geoNetwork.region',	'geoNetwork.metro',	'geoNetwork.city','geoNetwork.networkDomain',
     'totals.hits', 'totals.pageviews', 'totals.bounces','totals.newVisits', # 'bounces' = 이탈여부, 'newVisits' = 재방문여부
     'totals.sessionQualityDim','totals.transactions','totals.transactionRevenue','totals.totalTransactionRevenue', # qualitydim =활동점수, transaction=구매활동횟수,transactionRevenue=비용제거거래액,totals.totalTransactionRevenue=비용포함
     'totals.timeOnSite',  # timeonsite=사이트머문시간
     'trafficSource.campaign', 'trafficSource.adContent', # campaign=캠페인, adContent=캠패인 안의 컨텐츠
     'trafficSource.source','trafficSource.referralPath', # source=유입사이트, referralPath =넘어 온 사이트(nan이면 사이틑에 직접들어온것) 
     'trafficSource.medium','trafficSource.keyword',      # medium=유입매체, keyword=유입검색단어, 
     'trafficSource.adwordsClickInfo.gclId','trafficSource.adwordsClickInfo.adNetworkType', # gclId=구글ad로 유입(아닌 경우Nan), adNetworkType=광고타입 googlesarh(검색결과),GDN(타사이트 광고)
       ]]